In [12]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from env import github_token, github_username
import acquire as a


In [2]:
pageCount = 15 
urlsList = []
base = "https://github.com/search?o=desc&p={}&q=shoes&s=stars&type=Repositories" #curly braces let you format


for x in range(pageCount)[0:]:
    urlsList.append(base.format(x))

In [3]:
urlsList

['https://github.com/search?o=desc&p=0&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=1&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=2&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=3&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=4&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=5&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=6&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=7&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=8&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=9&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=10&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=11&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=12&q=shoes&s=stars&type=Repositories',
 'https://github.com/s

In [4]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception("You need to follow the instructions marked TODO in this script before trying to use it")

In [8]:
def get_repo_links(github_token=github_token, github_username=github_username,
                                         topic='shoes', number_of_pages=5):
    '''
    Takes in a topic, your unique github API token, and your github username as
    strings and an interger for the number of pages to query
    Returns: list of repositories from GitHub in the form of
    '<username>/<repo_name>'
    '''
    # set URL without page number
    url = f'https://github.com/topics/{topic}?&s=stars&page='
    # set header for github auth
    headers = {"Authorization": f"token {github_token}",
               "User-Agent": github_username}
    # set empty list for total repos scraped
    list_of_repos = []
    # for each page in range of provided number
    for i in range(1, number_of_pages + 1):
        # obtain page data
        time.sleep(15)
        response = requests.get(url + str(i), headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.find_all('h3')
        # set empty list to agg for each page
        page_list = []
        for repo in content:
            # confirms that h3 tag contains repo information
            if [re.search(r'(\S+)', x.text).group(1) for x in repo.find_all('a')] != []:
                # returns owner username and repo name
                user_name, repo_name = [re.search(r'(\S+)', x.text)\
                                         .group(1) for x in repo.find_all('a')]
                # add to page list for each repo on page
                page_list.extend([f'{user_name}/{repo_name}'])
        # add new page list into existing list for total repos
        list_of_repos.extend(page_list)
        print(f'Page {i} Completed')
    # saves returned list into .py file for calling in later functions
    with open("repos.py", "w") as repos:
        repos.write(f'REPOS = {list_of_repos}')
    return list_of_repos
    

In [9]:
repo_list = get_repo_links()

Page 1 Completed
Page 2 Completed
Page 3 Completed
Page 4 Completed
Page 5 Completed


In [10]:
len(repo_list)

98

In [11]:
repo_list

['shoes/shoes4',
 'druv5319/Sneaks-API',
 'shoes/shoes3',
 'WebAR-rocks/WebAR.rocks.hand',
 'brandontruggles/Selenium-Shoe-Bot-NakedCPH',
 'dilip-dawadi/shoeStore',
 'developer-junaid/Nike-Shoe-Store',
 'alisonmonteiro/shoe-size-converter',
 'LucaArgentieri/Shoe-Discord-Bot',
 'antonjlin/adidas-account-generator',
 'Fyko/stockx-gif',
 'iissh/TrendStop',
 'Hetessy/Shoes-Store-App-UI-Flutter',
 'Rhelli/Shoes-Github-API',
 'kenmoini/multiverse-of-multicluster-madness',
 'aravind2060/Capstone0',
 'daxidngyn/stockx-data',
 'adityar224/sneakerx',
 'fumixia/supreme-api-ts',
 'JDsnyke/UP-Flow',
 'JordanAssayah/shoe-order',
 'amouliom/sneakers',
 'harryleevn93/shoes-shop',
 'mprabs/Shoe-UI-mockUp',
 'MaximKuklin/3D_CenterNet',
 'Ohohko/shoes',
 'tyrue/3th_Engineering-design2',
 'udl/shoesql',
 'Nickwang3/GearGuide',
 'DanielMafra/desafio-1-papodedev',
 'Jolonte/challenge-codelandia-jordan-shoes',
 'Spitfire5720/is-the-shoe-for-you-.github.io',
 'lucianojunnior17/nike-site',
 'KeyulJain/RishitBl